### Data Analyst Assignment Step2

Evaluate RSSI threshold

1. Using the client stats sample data provided to you calculate what % of devices are connected 2.4 GHz and 5 

2. As per the given conditions, (Mentioned in pdf), Calculate what % of devices that will be on 2.4GHz and 5GHz using as per condition. Will we gain more 5GHz?


In [1]:
# load the librabries
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import count, col, when

In [2]:
# Create a Spark context
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(spark)

In [3]:
## Loading the data form part2 client sample amd reading it to a dataframe
df = sqlContext.read.format("csv").option("header","true").load('client_stats_sample_0225part2.csv')

In [4]:
# how many rows are there in dataset
df.count()

350596

In [5]:
# What is the schema of dataframe
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- location_id: string (nullable = true)
 |-- client_mac: string (nullable = true)
 |-- freq_band: string (nullable = true)
 |-- mac_address: string (nullable = true)
 |-- country: string (nullable = true)
 |-- vendor_name: string (nullable = true)
 |-- rssi_percs_25: string (nullable = true)
 |-- rssi_percs_75: string (nullable = true)
 |-- isp: string (nullable = true)
 |-- total_bytes_tx: string (nullable = true)
 |-- total_bytes_rx: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- txWeightedPhyRate: string (nullable = true)
 |-- rxWeightedPhyRate: string (nullable = true)
 |-- idtype: string (nullable = true)
 |-- is_static: string (nullable = true)
 |-- weighted_prr_tx: string (nullable = true)
 |-- weighted_prr_rx: string (nullable = true)
 |-- num_samples: string (nullable = true)
 |-- is_connected_to_pod: string (nullable = true)
 |-- pod_serial: string (nullable = true)
 |-- control_group: string (nullable = true)
 |-- b

In [6]:
# _c0 is a noise in the data, so we need to drop it
df =df.drop("_c0")

In [7]:
# Are there null values in dataset. Which columns has how many null values?
# this is very important 
null_value_counts = {i:df.filter(df[i].isNull()).count() for i in df.columns}
null_value_counts

{'bandCap2G': 21287,
 'bandCap5G': 21287,
 'client_mac': 0,
 'control_group': 330687,
 'country': 859,
 'freq_band': 0,
 'idtype': 22465,
 'isBTMSupported': 21287,
 'is_connected_to_pod': 0,
 'is_static': 222911,
 'isp': 859,
 'location_id': 0,
 'mac_address': 0,
 'maxChWidth': 21287,
 'maxMcs': 21287,
 'maxNss': 21287,
 'num_samples': 0,
 'phyMode': 21287,
 'pod_serial': 350246,
 'rssi_percs_25': 305,
 'rssi_percs_75': 305,
 'rxWeightedPhyRate': 5183,
 'total_bytes_rx': 5183,
 'total_bytes_tx': 14375,
 'ts': 0,
 'txWeightedPhyRate': 14375,
 'vendor_name': 223524,
 'weighted_prr_rx': 5183,
 'weighted_prr_tx': 14375}

Yes there are many null values but lets decide later for imputation

#### Features of interest
We need to understand the relationship between RSSI and Freq_band but we need to also consider the devices (mac addresses for wifi and mac address for client (client can be any connected device such as phone, TV, laptop etc.) 


In [8]:
### Select features of interest for analysis as descbied in problem
df_select = df.select("mac_address", "client_mac","freq_band","rssi_percs_25")
df_select.show()

+--------------------+--------------------+---------+-------------+
|         mac_address|          client_mac|freq_band|rssi_percs_25|
+--------------------+--------------------+---------+-------------+
|5b969ea8efcc5d3e1...|81294f9c314119e79...|       5G|        -62.0|
|9cd4189fefcd878d5...|c544851fa0c7d8450...|       5G|        -37.0|
|9cd4189fefcd878d5...|c544851fa0c7d8450...|       5G|        -37.0|
|9cd4189fefcd878d5...|c544851fa0c7d8450...|       5G|        -37.0|
|9cd4189fefcd878d5...|c544851fa0c7d8450...|       5G|        -37.0|
|c6ecd62a6319f6790...|412adb9ac900c7125...|     2.4G|        -61.0|
|c6ecd62a6319f6790...|412adb9ac900c7125...|     2.4G|        -61.0|
|c6ecd62a6319f6790...|412adb9ac900c7125...|     2.4G|        -61.0|
|c6ecd62a6319f6790...|412adb9ac900c7125...|     2.4G|        -61.0|
|c897d58e4c62b048d...|5d3fb54faf8938363...|       5G|        -38.0|
|c897d58e4c62b048d...|5d3fb54faf8938363...|       5G|        -38.0|
|c897d58e4c62b048d...|5d3fb54faf8938363...|     

In [9]:
# Let us understand distribution of freq_band within the wifi mac
# Assumption : Mac address is for eifi device
df.stat.crosstab("mac_address", "freq_band").show()

+---------------------+----+---+
|mac_address_freq_band|2.4G| 5G|
+---------------------+----+---+
| f623bb4e5415c487b...| 240| 84|
| 05a7c9001714c2215...| 145|465|
| 97c19d8021d8c248a...| 121|114|
| 07d0ed1e38cf230a9...|  94|273|
| 956470c5c6b408571...| 398|580|
| 7dc74fb0d808e98b3...| 174|123|
| 824fecede0044b33f...| 736|381|
| 198591b836dbdb12b...|  51| 45|
| 51a12a54c04087ddf...| 278|279|
| 1e31684fd98d43901...| 147|253|
| 92c4528559e7c6ed5...|  75|275|
| 679be718426c02f63...| 126|125|
| 1a0c27c5ef0dbac69...| 451|339|
| b3499d63cf64ba727...| 518|164|
| 8157d52bf2fe92f79...|  78|244|
| 5f1fc100d3bd50eab...|   0| 42|
| 02285fd68e5bd179a...| 402| 48|
| fa4ac56da9097c73d...|   0| 96|
| a8dc46dda36ca21f5...| 134|225|
| 24d3ea68d8ef89770...| 249|175|
+---------------------+----+---+
only showing top 20 rows



In [10]:
#  Let us understand distribution of freq_band within the client mac
df.stat.crosstab("client_mac", "freq_band").show()

+--------------------+----+---+
|client_mac_freq_band|2.4G| 5G|
+--------------------+----+---+
|6c108075386b9bac5...|   0| 13|
|926b265569cc677b8...|  52|  0|
|64aaa39a634c845c3...|   4|  0|
|e85310fe4c4d96b7a...|   0| 48|
|9694df765bee22177...|   4| 35|
|c92150b6204bb09a1...|   0| 48|
|991f0219bf779f92c...|   0| 48|
|49e4a625381b85b59...|  44|  0|
|770bf9fd4898249c9...|  48|  0|
|c5cb976ee8161e4e9...|   0| 12|
|7644580503a3cf67f...|  48|  0|
|03c829e8ae46efe67...|  52|  0|
|20c423411d4752114...|   0| 44|
|bebf74bee765ed470...|  48|  0|
|15b102383f5ab6ac0...|   0| 48|
|1151b29fa53a37304...|   9| 43|
|468d4950090d9606c...|   0| 52|
|40cd4df24ef98b291...|   0| 48|
|3cced5d8859dea268...|   1|  7|
|ae933e3bbd46c58cc...|   0| 19|
+--------------------+----+---+
only showing top 20 rows



## Note
There are conditions where one mac address is having entries for both 2.4G and 5G, so this is the case, where I have not considered the groupby with Mac. I am calculating freq_band Vs device % calculation directly


In [11]:
## Let us understand the disctribution of columns
for i in df_select.columns:
    print("This is the value counts for", i)
    df_select.groupBy(i).count().show(5)

This is the value counts for mac_address
+--------------------+-----+
|         mac_address|count|
+--------------------+-----+
|8ac49834a5054c6e3...|  757|
|d574d684eef28cd14...|  557|
|2310fafb4924fda2b...|  534|
|576beeed3bdadb939...|  130|
|7d5c8ca69132eef8e...|  382|
+--------------------+-----+
only showing top 5 rows

This is the value counts for client_mac
+--------------------+-----+
|          client_mac|count|
+--------------------+-----+
|54ff4e897a6b8e66a...|   52|
|cba440e81f4959062...|   48|
|17a6673982853b3d5...|   48|
|ed066a83e420e9efa...|   52|
|2d8451e1f833f8b78...|   37|
+--------------------+-----+
only showing top 5 rows

This is the value counts for freq_band
+---------+------+
|freq_band| count|
+---------+------+
|     2.4G|159914|
|       5G|190682|
+---------+------+

This is the value counts for rssi_percs_25
+-------------+-----+
|rssi_percs_25|count|
+-------------+-----+
|        -93.0|    3|
|        -73.0| 6983|
|         -7.0|    2|
|        -18.0|  4

In [12]:
## Devices per freq_band
value_24g = df_select.filter(col("freq_band")=='2.4G').groupBy("freq_band").count()

# percetage of the 2.4G device in complete dataset
pct_24g = value_24g.select('count').first()[0]/df_select.count()
print("percetage of the 2.4G device in complete dataset", pct_24g)

percetage of the 2.4G device in complete dataset 0.45612043491654214


In [13]:
## Devices per freq_band
value_5g = df_select.filter(col("freq_band")=='5G').groupBy("freq_band").count()

# percetage of the 2.4G device in complete dataset
pct_5g = value_5g.select('count').first()[0]/df_select.count()
print("percetage of the 2.4G device in complete dataset", pct_5g)

percetage of the 2.4G device in complete dataset 0.5438795650834579


### Step2 - Implement the new model condition for the freq_band

In [14]:
# Are there null values in dataset. Which columns has how many null values
# this is very important 
null_value_counts = {i:df_select.filter(df[i].isNull()).count() for i in df_select.columns}
null_value_counts

{'client_mac': 0, 'freq_band': 0, 'mac_address': 0, 'rssi_percs_25': 305}

### Note
There are missing values in rssi_percs_25 column. We need to remove null values as it is going to impact the calculated values


In [15]:
# drop the null values
df_select = df_select.dropna(how='any')

In [16]:
# calculate the values and assigned to new columns
final_df = df_select.\
        withColumn("new_freq_band", when((col("rssi_percs_25") >= -60 ) & (col("freq_band")=='2.4G'), "5G").\
        when((col("rssi_percs_25") <= -75 ) & (col("freq_band")=='5G'), "2.4G").\
                   when((col("rssi_percs_25") > -75 ) & (col("freq_band")=='5G'), "5G").\
                   when((col("rssi_percs_25") <= -60 ) & (col("freq_band")=='2.4G'), "2.4G").\
                    otherwise("ERROR"))
    
## Error if any mistake in the dataset

In [17]:
final_df.show(10)

+--------------------+--------------------+---------+-------------+-------------+
|         mac_address|          client_mac|freq_band|rssi_percs_25|new_freq_band|
+--------------------+--------------------+---------+-------------+-------------+
|5b969ea8efcc5d3e1...|81294f9c314119e79...|       5G|        -62.0|           5G|
|9cd4189fefcd878d5...|c544851fa0c7d8450...|       5G|        -37.0|           5G|
|9cd4189fefcd878d5...|c544851fa0c7d8450...|       5G|        -37.0|           5G|
|9cd4189fefcd878d5...|c544851fa0c7d8450...|       5G|        -37.0|           5G|
|9cd4189fefcd878d5...|c544851fa0c7d8450...|       5G|        -37.0|           5G|
|c6ecd62a6319f6790...|412adb9ac900c7125...|     2.4G|        -61.0|         2.4G|
|c6ecd62a6319f6790...|412adb9ac900c7125...|     2.4G|        -61.0|         2.4G|
|c6ecd62a6319f6790...|412adb9ac900c7125...|     2.4G|        -61.0|         2.4G|
|c6ecd62a6319f6790...|412adb9ac900c7125...|     2.4G|        -61.0|         2.4G|
|c897d58e4c62b04

In [18]:
# Old model freq_band
final_df.groupBy("freq_band").count().show()

+---------+------+
|freq_band| count|
+---------+------+
|     2.4G|159668|
|       5G|190623|
+---------+------+



In [19]:
# New model freq_band
final_df.groupBy("new_freq_band").count().show()

+-------------+------+
|new_freq_band| count|
+-------------+------+
|         2.4G|105754|
|           5G|244537|
+-------------+------+



### Caluclate the % value of each category

In [20]:
## Devices per freq_band
value_24g = final_df.filter(col("freq_band")=='2.4G').groupBy("freq_band").count()
pct_24g = value_24g.select('count').first()[0]/final_df.count()
print("percentage of the 2.4G device in complete dataset", pct_24g)

percentage of the 2.4G device in complete dataset 0.45581530784404967


In [21]:
## Devices per freq_band
value_5g = final_df.filter(col("freq_band")=='5G').groupBy("freq_band").count()
pct_5g = value_5g.select('count').first()[0]/final_df.count()
print("percentage of the 5G device in complete dataset", pct_5g)

percentage of the 5G device in complete dataset 0.5441846921559503


In [22]:
## Devices per freq_band
new_model_value_24g = final_df.filter(col("new_freq_band")=='2.4G').groupBy("new_freq_band").count()
new_model_pct_24g = new_model_value_24g.select('count').first()[0]/final_df.count()

print("percentage of the 2.4G device in New model dataset", new_model_pct_24g)

percentage of the 2.4G device in New model dataset 0.3019032747058874


In [23]:
## Devices per freq_band
new_model_value_5g = final_df.filter(col("new_freq_band")=='5G').groupBy("new_freq_band").count()
new_model_pct_5g = new_model_value_5g.select('count').first()[0]/final_df.count()\

print("percentage of the 5G device in New model dataset", new_model_pct_5g)

percentage of the 5G device in New model dataset 0.6980967252941126


### Conclusion:




1. Percetage of the 5G device in New model dataset 0.6980967252941126  
2. Percetage of the 2.4G device in New model dataset 0.3019032747058874

5G devices are expected to increase significantly in this case. 
The results are influnced with assumption made during the analysi and this is consider as basic analysis on the data. So, we can farther enhance the analysis with more discussions, SMEs help and better approaches  

### Next Steps

    I need to understand more about the dataset and mapping.
    Is there instance mapping when user is connecting multiple times in a day to wifi with different freq_band?
    TV and Home appliance could be static while mobile and laptop can have dynamic connectivity as per user availability, Need to understand more these points


### There are duplicate/multiple entries for 2.4GHz and 5Ghz for single Mac Address.

    Mac addresses are unique to devices, so calculating the % of device truly connected with right freq_band needs more information.
    Mac_address : This seems to be mac address of Wi-Fi device which will have signal for 5GHz and 2.4GHz so we will not use this feature for device % calculation
    client_mac : This seems to be client device (ex. phone, laptop, tv) which will have connection for 5GHz and 2.4GHz so we will group by this feature for device % calculation
